<a href="https://colab.research.google.com/github/Miranda-Merino-Irigoyen-Jorge-Missael/Simulaci-n-2/blob/main/Proyecto_Ingenieria_Financiera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparación del entorno.


In [ ]:


# Instalar webdriver-manager si no está instalado
!pip install webdriver-manager selenium

# Importar librerías necesarias
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Configurar opciones del navegador
options = Options()
options.add_argument("--headless")  # Sin interfaz gráfica
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Descargar automáticamente la versión compatible de ChromeDriver
service = Service(ChromeDriverManager().install())

# Crear el navegador
driver = webdriver.Chrome(service=service, options=options)

# Probar conexión
driver.get("https://www.google.com")
print("Título de la página:", driver.title)

# Cerrar navegador
driver.quit()


Título de la página: Google


# Comparador de precios

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font
from openpyxl.utils.dataframe import dataframe_to_rows

# Configurar ScraperAPI
api_key = "d5fc8be171b2e7e86724a89d7e05f8da"
amazon_url = f"https://www.amazon.com/s?k=laptop"
amazon_scraper_url = f"http://api.scraperapi.com?api_key={api_key}&url={amazon_url}"

mercadolibre_url = "https://listado.mercadolibre.com.mx/laptop"

# Función para extraer productos de Amazon
def fetch_amazon_products():
    response = requests.get(amazon_scraper_url)
    products = []

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        product_titles = soup.find_all('h2', class_='a-size-medium a-spacing-none a-color-base a-text-normal')
        price_spans = soup.find_all('span', class_='a-price')

        product_names = [title.text.strip() for title in product_titles]
        product_prices = [price.find('span', class_='a-offscreen').text.strip() for price in price_spans if price.find('span', class_='a-offscreen')]

        products = [{"Name": name, "Price": price, "Source": "Amazon"} for name, price in zip(product_names, product_prices)][:20]

    return products

# Función para extraer productos de MercadoLibre
def fetch_mercadolibre_products():
    response = requests.get(mercadolibre_url)
    products = []

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        product_items = soup.find_all('li', class_='ui-search-layout__item shops__layout-item', limit=20)

        for item in product_items:
            name_tag = item.find('a', class_='poly-component__title')
            price_tag = item.find('span', class_='andes-money-amount__fraction')

            name = name_tag.text.strip() if name_tag else "Nombre no encontrado"
            price = f"${price_tag.text.strip()}" if price_tag else "Precio no encontrado"
            products.append({"Name": name, "Price": price, "Source": "MercadoLibre"})

    return products

# Extraer productos
amazon_products = fetch_amazon_products()
mercadolibre_products = fetch_mercadolibre_products()

# Combinar productos
all_products = amazon_products + mercadolibre_products

# Crear DataFrame
df = pd.DataFrame(all_products)

# Crear archivo Excel
wb = Workbook()
ws = wb.active
ws.title = "Productos"

# Agregar datos de DataFrame a Excel
for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), start=1):
    for c_idx, value in enumerate(row, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Aplicar estilos
header_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
amazon_fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")
mercadolibre_fill = PatternFill(start_color="98FB98", end_color="98FB98", fill_type="solid")
border = Border(left=Side(style="thick"),
                right=Side(style="thick"),
                top=Side(style="thick"),
                bottom=Side(style="thick"))

# Estilizar encabezados
for cell in ws[1]:
    cell.fill = header_fill
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center", vertical="center")

# Estilizar filas
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        source = ws.cell(row=cell.row, column=3).value
        if source == "Amazon":
            cell.fill = amazon_fill
        elif source == "MercadoLibre":
            cell.fill = mercadolibre_fill

# Agregar bordes a toda la tabla
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        cell.border = border

# Guardar archivo
wb.save("productos_estilizados.xlsx")

print("Archivo 'productos_estilizados.xlsx' creado con éxito.")


Archivo 'productos_estilizados.xlsx' creado con éxito.
